# Coffee Object Detection: Single Image Extraction

<!-- <img src="https://pages.databricks.com/rs/094-YMS-629/images/SBUX-DB.png"/> -->

This notebook provides a simplified example of object detection (e.g. coffee, cups, chair, etc.) within images and videos.  In addition to use *Databricks Runtime for ML - GPU*, we are also working with the [Image AI](https://github.com/OlafenwaMoses/ImageAI/releases/download/2.0.2/imageai-2.0.2-py3-none-any.whl) library.

## Setup
There are additional installations to include the **pre-trained** models as part of **[ImageAI](https://github.com/OlafenwaMoses/ImageAI/blob/master/imageai/Detection/VIDEO.md):**
* **Yolo V3**: You only look once (YOLO) is a state-of-the-art, real-time object detection system; we are using V3 as noted in [YOLOv3: An Incremental Improvement](https://pjreddie.com/media/files/papers/YOLOv3.pdf) (using this model for images)
* **Resnet**: Residual Network model (in h5 format) that we are using for feature extraction (using this model for videos)

In [0]:
%sh
## Install the ImageAI pre-trained models: YoloV3, ResNet
# wget -P /dbfs/ml/sbux/cafe/ https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/yolo.h5
# wget -P /dbfs/ml/sbux/cafe/ https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/resnet50_coco_best_v2.0.1.h5

In [0]:
# Uncomment this to copy the necessary files for notebook 1 and 3
#dbutils.fs.cp("/mnt/tardis6/cafe/", "/ml/sbux/cafe/sources/", recurse=True)

In [0]:
## ------------------------------
## Miscellaneous Functions
## ------------------------------
# displayVid(): Shows video from mounted cloud storage
def displayVid(filepath):
  return displayHTML("""
  <video width="480" height="320" controls>
  <source src="/files/%s" type="video/mp4">
  </video>
  """ % filepath)

# displayDbfsVid(): Shows video from DBFS
def displayDbfsVid(filepath):
  return displayHTML("""
  <video width="480" height="320" controls>
  <source src="/dbfs/%s" type="video/mp4">
  </video>
  """ % filepath)

# displayImg(): Shows image from dbfs/cloud storage
def displayImg(filepath):
  dbutils.fs.cp(filepath, "FileStore/%s" % filepath)
  return displayHTML("""
  <img src="/files/%s" width="800">
  """ % filepath)

In [0]:
# Configure TensorFlow for Auto-Growth
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.visible_device_list = "0"
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.5

tf.Session(config=config)
tf.set_random_seed(42) # For reproducibility

# Configure MLflow Experiment
#mlflow_experiment_id = 2102416

## Extracting Coffee Drinks from a Single Image

We are using the following image of Starbucks iced coffees and using the YoloV3 model to extract out objects.

<img src="https://pages.databricks.com/rs/094-YMS-629/images/five_drinks.jpg" width="600"/>

<!-- 
# Can display images from `FileStore` 
displayImg("/FileStore/cafe/source_images/five_drinks.jpg")
-->

In [0]:
# Import ObjectDetection from ImageAI
from imageai.Detection import ObjectDetection
import os
import h5py

# Configure source images and execution paths
execution_path = "/dbfs/ml/sbux/cafe/"
execution_path_source = "/dbfs/ml/sbux/cafe/sources/"
execution_path_results = "/dbfs/ml/sbux/cafe/"

# Use ObjectDetection using the YoloV3 model
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join(execution_path , "yolo.h5"))
detector.loadModel()

# Detect Objects in the image
detections, objects_path = detector.detectObjectsFromImage(
  input_image=os.path.join(execution_path_source, "five_drinks.jpg"), 
  output_image_path=os.path.join(execution_path_results, "results.jpg"), 
  minimum_percentage_probability=30,  
  extract_detected_objects=True
)

# Print out the images extracted, box points, and percentage probability
for eachObject, eachObjectPath in zip(detections, objects_path):
    print(eachObject["name"] , " : " , eachObject["percentage_probability"], " : ", eachObject["box_points"] )
    print("Object's image saved in " + eachObjectPath)
    print("--------------------------------")

dining table : 32.902032136917114 : (118, 177, 508, 512)
Object's image saved in /dbfs/ml/sbux/cafe/results.jpg-objects/dining table-1.jpg
--------------------------------
cup : 44.94319260120392 : (399, 106, 629, 451)
Object's image saved in /dbfs/ml/sbux/cafe/results.jpg-objects/cup-2.jpg
--------------------------------
cup : 74.68961477279663 : (800, 114, 1066, 167)
Object's image saved in /dbfs/ml/sbux/cafe/results.jpg-objects/cup-3.jpg
--------------------------------
cup : 98.43920469284058 : (828, 101, 1042, 492)
Object's image saved in /dbfs/ml/sbux/cafe/results.jpg-objects/cup-4.jpg
--------------------------------
cup : 99.2410659790039 : (536, 165, 888, 527)
Object's image saved in /dbfs/ml/sbux/cafe/results.jpg-objects/cup-5.jpg
--------------------------------
cup : 99.66422915458679 : (181, 137, 535, 528)
Object's image saved in /dbfs/ml/sbux/cafe/results.jpg-objects/cup-6.jpg
--------------------------------
cup : 99.67660307884216 : (1015, 141, 1303, 526)
Object's image saved in /dbfs/ml/sbux/cafe/results.jpg-objects/cup-7.jpg
--------------------------------

#### Observation

* Notice how the probability of the dining table is 32.9% while the probability of the last cup is 99.68%.  
* After the probability, there is also the box points (the square location in the image)

### View the Extracted Images

* The images are stored in the `/dbfs/ml/sbux/cafe/results.jpg-objects/` folder
* Let's view them using the `ImageSchema` method included in Apache Spark 2.4; more information can be found at [Introducing Built-in Image Data Source in Apache Spark 2.4](https://databricks.com/blog/2018/12/10/introducing-built-in-image-data-source-in-apache-spark-2-4.html).

In [0]:
%fs ls /ml/sbux/cafe/results.jpg-objects/

path,name,size
dbfs:/ml/sbux/cafe/results.jpg-objects/cup-2.jpg,cup-2.jpg,8534
dbfs:/ml/sbux/cafe/results.jpg-objects/cup-3.jpg,cup-3.jpg,2066
dbfs:/ml/sbux/cafe/results.jpg-objects/cup-4.jpg,cup-4.jpg,8843
dbfs:/ml/sbux/cafe/results.jpg-objects/cup-5.jpg,cup-5.jpg,16740
dbfs:/ml/sbux/cafe/results.jpg-objects/cup-6.jpg,cup-6.jpg,15149
dbfs:/ml/sbux/cafe/results.jpg-objects/cup-7.jpg,cup-7.jpg,14760
dbfs:/ml/sbux/cafe/results.jpg-objects/dining table-1.jpg,dining table-1.jpg,13412


In [0]:
from pyspark.ml.image import ImageSchema

extractedImages = ImageSchema.readImages("/ml/sbux/cafe/results.jpg-objects")
display(extractedImages)

image
"List(dbfs:/ml/sbux/cafe/results.jpg-objects/cup-2.jpg, 345, 230, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/ml/sbux/cafe/results.jpg-objects/cup-5.jpg, 362, 352, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/ml/sbux/cafe/results.jpg-objects/cup-6.jpg, 391, 354, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/ml/sbux/cafe/results.jpg-objects/cup-7.jpg, 385, 288, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/ml/sbux/cafe/results.jpg-objects/dining table-1.jpg, 335, 390, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"


## Roadmap
We are starting with ImageAI due to the existing pre-trained models. Future plans include:
* Using `keras-yolov3` directly so we can train our own models to be more Starbucks specific (coffee, baked goods, cups, etc.) 
* Scale up to improve the speed of these models
* Scale out to handle more images and videos in a *distributed* fashion